This code was adapted from:


*   https://www.geeksforgeeks.org/spelling-checker-in-python/
*   NLP Labs


In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from numpy import array
from string import punctuation
from os import listdir
from collections import Counter
from nltk.corpus import stopwords
from pandas import DataFrame
from matplotlib import pyplot
import random
import nltk
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import rand
from sklearn.metrics.pairwise import cosine_similarity
from numpy import argsort
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from collections import Counter
from nltk.stem.porter import PorterStemmer
#from spellchecker import SpellChecker
nltk.download('punkt')
nltk.download('stopwords')

drive.mount("/content/drive")

data_path = "/content/drive/My Drive/Colab Notebooks/21COP509/LabDatasets/"
!ls "/content/drive/My Drive/Colab Notebooks/21COP509/LabDatasets/"

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 ArtsRatings_5000_test.txt	      'My CW 2.ipynb'
 ArtsRatings_5000_train.txt	      'MY CW 3.ipynb'
 ArtsReviews_5000_test.txt	      'My CW 4.ipynb'
 ArtsReviews_5000_train.txt	       Reduced_ArtsRatings_5000.txt
 glove.6B.100d.txt		       Reduced_ArtsReviews_5000.txt
 JewelleryReviewsLSA.csv	       review_polarity
 JewelleryReviewsQueryRelevantID.csv  'Task 3.ipynb'
 metamorphosis_clean.txt	       Untitled0.ipynb
'My CW1.ipynb'


In [ ]:
# Download SummerTime 
# Swith to the Summertime directory

!git clone https://github.com/Yale-LILY/SummerTime.git
%cd SummerTime/

Cloning into 'SummerTime'...
remote: Enumerating objects: 4381, done.
remote: Counting objects: 100% (2209/2209), done.
remote: Compressing objects: 100% (1135/1135), done.
remote: Total 4381 (delta 1486), reused 1523 (delta 933), pack-reused 2172
Receiving objects: 100% (4381/4381), 9.87 MiB | 26.40 MiB/s, done.
Resolving deltas: 100% (2352/2352), done.
/content/SummerTime


In [ ]:
# Pip install Summertime locally

!pip install -e .

Obtaining file:///content/SummerTime
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 69 kB 5.6 MB/s 
     |████████████████████████████████| 67 kB 5.6 MB/s 
     |████████████████████████████████| 1.5 MB 51.1 MB/s 
     |████████████████████████████████| 62.5 MB 1.2 MB/s 
     |████████████████████████████████| 12.8 MB 39.3 MB/s 
     |████████████████████████████████| 156 kB 45.8 MB/s 
     |████████████████████████████████| 255 kB 55.7 MB/s 
     |████████████████████████████████| 10.6 MB 41.6 MB/s 
     |████████████████████████████████| 69 kB 6.6 MB/s 
     |████████████████████████████████| 64 kB 2.6 MB/s 
     |████████████████████████████████| 2.1 MB 52.3 MB/s 
     |████████████████████████████████| 221 kB 55.3 MB/s 
     |████████████████████████████████| 1.2 MB 49.0 MB/s 
     |████████████████████████████████| 24.2 MB 1.4 MB/s 
     |█████████████████████████

In [ ]:
## Finish setup

# Setup ROUGE (needed to use ROUGE evaluation metric)
!export ROUGE_HOME=/usr/local/bin/python/dist-packages/summ_eval/ROUGE-1.5.5/
!pip install -U  git+https://github.com/bheinzerling/pyrouge.git

  Cloning https://github.com/bheinzerling/pyrouge.git to /tmp/pip-req-build-f3ezl3ml
  Running command git clone -q https://github.com/bheinzerling/pyrouge.git /tmp/pip-req-build-f3ezl3ml
  Created wheel for pyrouge: filename=pyrouge-0.1.3-py3-none-any.whl size=191924 sha256=c902b6bde21a757b896865b114de4bbfda3723f426e617912ef5a2ce206caeda
  Stored in directory: /tmp/pip-ephem-wheel-cache-awx82g_e/wheels/2f/1c/55/4e264b4e45f2a6f39a50e95cff45e57b54f202db830f125bb2
Successfully built pyrouge


In [ ]:
# If you've been prompted to restart the kernel in either of the two cells above,
# Please do so
# Then run this cell to go back to the relevant directory

%cd /content/SummerTime/

/content/SummerTime


In [ ]:
# import modules for this notebook

from pprint import pprint
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# !pip install en_core_web_sm==3.0.0
!python -m spacy download en_core_web_sm

     |████████████████████████████████| 13.7 MB 22.3 MB/s 
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 2.2.5
    Uninstalling en-core-web-sm-2.2.5:
      Successfully uninstalled en-core-web-sm-2.2.5
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/21COP509/LabDatasets/JewelleryReviewsLSA.csv')
reviews = df["Reviews"].to_numpy()
ratings = df["Ratings"].to_numpy()


In [ ]:

corpus1 = list()
corpus2 = list()
corpus3 = list()
corpus4 = list()
corpus5 = list()



for i in range(200):
  #print(i)
  if(ratings[i]==1):
    corpus1.append(reviews[i])

  if(ratings[i]==2):
    corpus2.append(reviews[i])

  if(ratings[i]==3):
    corpus3.append(reviews[i])

  if(ratings[i]==4):
    corpus4.append(reviews[i])

  if(ratings[i]==5):
    corpus5.append(reviews[i])   

In [ ]:
print(len(corpus1)+len(corpus2)+len(corpus3)+len(corpus4)+len(corpus5))

200


In [ ]:
print(len(corpus1))

23


In [ ]:
pip install scipy

In [ ]:
from summertime import model

In [ ]:
lexrank = model.LexRankModel(corpus1)

# Inference
summary = lexrank.summarize(corpus1)
for i in range(len(summary)):
  print("Summary Review %d: "%(i+1), summary[i])

Summary Review 1:  i expect like regular size of ring,but this one look like a ring for toy or something funy, the MM of our rings is 5MM and this ring may be is 1MMso ridiculousMartin1/5 ct.tw Round Diamond Solitaire Ring in 18k White Gold
Summary Review 2:  Pick one! Is it a wedding band, toe ring or thumb ring?
Summary Review 3:  And this time 1 diamond missing on the shoulder of the ring. I returned the ring for a replacement of the same ring, the second ring was even worse.
Summary Review 4:  And this time 1 diamond missing on the shoulder of the ring. I returned the ring for a replacement of the same ring, the second ring was even worse.
Summary Review 5:  I seriously not wasting your money on this low quality item This is a huge waste of money I had it for about a month when the part that holds the symbol to the necklace broke.
Summary Review 6:  Very flimsy.I would not recommend this item The quality and look were not what I had anticipated.
Summary Review 7:  If I saw this ite

In [ ]:
from summertime.evaluation import SUPPORTED_EVALUATION_METRICS

print(SUPPORTED_EVALUATION_METRICS)

import summertime.evaluation as st_eval

bert_eval = st_eval.BertScore()

from summertime import dataset, model
from pprint import pprint
from summertime.evaluation import BertScore
from summertime.evaluation import Meteor
from summertime.evaluation import Bleu

targets = ['I will not be buying this ring again.',' I will not be ordering from Amazon or this jeweler in the future.', 'The first shipment of this ring had scratches on the diamond.' , 'Very disappointed in the appearance and quality.']

summaries = summary

[<class 'summertime.evaluation.bertscore_metric.BertScore'>, <class 'summertime.evaluation.bleu_metric.Bleu'>, <class 'summertime.evaluation.rouge_metric.Rouge'>, <class 'summertime.evaluation.rougewe_metric.RougeWe'>, <class 'summertime.evaluation.meteor_metric.Meteor'>]


In [ ]:
# Calculate BertScore
bert_metric = BertScore()
bert_metric.evaluate(summaries[:4], targets)


hash_code: bert-base-uncased_L8_no-idf_version=0.3.11(hug_trans=4.5.1)


{'bert_score_f1': 0.4517529606819153}

In [ ]:
# Calculate Meteor score
meteor_metric = Meteor()
meteor_metric.evaluate(summaries[:4], targets)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


{'meteor': 0.06018583400876429}

In [ ]:
# Calculate BleuScore
bleu_metric = Bleu()
bleu_metric.evaluate(summaries[:4], targets)

{'bleu': 0.011218874467832134}

Summarizing and evaluating class 2

In [ ]:
lexrank = model.LexRankModel(corpus2)

# Inference
summary = lexrank.summarize(corpus2)
for i in range(len(summary)):
  print("Summary Review %d: "%(i+1), summary[i])

Summary Review 1:  I wanted to know if this ring is like 2 rings in one, because this ring is beyond gorgeous, I just love it.
Summary Review 2:  It was not worth returning it as you would pay more for postage than what the ring cost. Nice ring for the an inexpensive ring, but one stone was missing.
Summary Review 3:  wore this toe ring one day and POOF the stone was gone so it sits in a pile of other broken sterling silver rings and things
Summary Review 4:  I would not recommend if you want a nice 1/2 carat ring. Ring is way too small and looks like a toy when putting it on.
Summary Review 5:  Spend your money on a higher quality item. The metal is already tarnishing too.
Summary Review 6:  its what i wanted :) but its not my favorite piercing of mine but i have to wear the bioplast cuz i break out with certain metals
Summary Review 7:  I wear it UNDER my shirt since it does not compliment anything I wear. It serves the purpose, but it seemed to me that the image was a lot prettier a

In [ ]:
from summertime.evaluation import SUPPORTED_EVALUATION_METRICS

print(SUPPORTED_EVALUATION_METRICS)

import summertime.evaluation as st_eval

bert_eval = st_eval.BertScore()

from summertime import dataset, model
from pprint import pprint
from summertime.evaluation import BertScore
from summertime.evaluation import Meteor
from summertime.evaluation import Bleu

targets = ['It was not worth returning it as you would pay more for postage than what the ring cost.' , 'The ring was nice and looked like picture but had a crack in one Garnet and another one had a large chip.' , 'I would recommend you spend it a little bit more and buy a better looking ring.' ,  'I\'m very disappointed with amazon for the lack of description.']

summaries = summary

[<class 'summertime.evaluation.bertscore_metric.BertScore'>, <class 'summertime.evaluation.bleu_metric.Bleu'>, <class 'summertime.evaluation.rouge_metric.Rouge'>, <class 'summertime.evaluation.rougewe_metric.RougeWe'>, <class 'summertime.evaluation.meteor_metric.Meteor'>]


In [ ]:
# Calculate BertScore
bert_metric = BertScore()
bert_metric.evaluate(summaries[:4], targets)

hash_code: bert-base-uncased_L8_no-idf_version=0.3.11(hug_trans=4.5.1)


{'bert_score_f1': 0.44789665937423706}

In [ ]:
# Calculate Meteor score
meteor_metric = Meteor()
meteor_metric.evaluate(summaries[:4], targets)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


{'meteor': 0.056363437882077}

In [ ]:
# Calculate BleuScore
bleu_metric = Bleu()
bleu_metric.evaluate(summaries[:4], targets)

{'bleu': 0.006164252063182463}

Summarizing and evaluating class 3

In [ ]:
lexrank = model.LexRankModel(corpus3)

# Inference
summary = lexrank.summarize(corpus3)
for i in range(len(summary)):
  print("Summary Review %d: "%(i+1), summary[i])

Summary Review 1:  Thanks alot Amazon. I returned it for a refund.
Summary Review 2:  Thanks alot Amazon. I returned it for a refund.
Summary Review 3:  very nice small sized ring I can stack it with other rings for different looks
Summary Review 4:  I got this ring for my birthday and I love it, I cannot imagine a woman not adoring this ring.
Summary Review 5:  I purchased the ring as I have a few nice gold necklaces I wear and I wanted a ring to work with them all. The ring doesn't look like a $35 ring.
Summary Review 6:  Comfortable ring. I wear the ring on my index finger and it fits great!
Summary Review 7:  Thank you! We appreciate the care and quick receiving of the ring.
Summary Review 8:  I love the ring its not a ring that you would want to wear as an everyday ring the claws that hold the stone in are very sharp and get stuck on things and have scratched me
Summary Review 9:  It's a cute ring so I just kept it for myself. I bought this ring as a Valentine's gift but couldn't 

In [ ]:
from summertime.evaluation import SUPPORTED_EVALUATION_METRICS

print(SUPPORTED_EVALUATION_METRICS)

import summertime.evaluation as st_eval

bert_eval = st_eval.BertScore()

from summertime import dataset, model
from pprint import pprint
from summertime.evaluation import BertScore
from summertime.evaluation import Meteor
from summertime.evaluation import Bleu

targets = ['This ring is a beautiful ring but the first shipment of this ring had scratches on the diamond and was missing 2 diamonds.' , 'I was not sure if the diamonds were actually blue or not.' , 'I was a little disappointed with the size of the rings.' ,  'Only thing is you cant have this ring resized.']

summaries = summary

[<class 'summertime.evaluation.bertscore_metric.BertScore'>, <class 'summertime.evaluation.bleu_metric.Bleu'>, <class 'summertime.evaluation.rouge_metric.Rouge'>, <class 'summertime.evaluation.rougewe_metric.RougeWe'>, <class 'summertime.evaluation.meteor_metric.Meteor'>]


In [ ]:
# Calculate BertScore
bert_metric = BertScore()
bert_metric.evaluate(summaries[:4], targets)

hash_code: bert-base-uncased_L8_no-idf_version=0.3.11(hug_trans=4.5.1)


{'bert_score_f1': 0.43645626306533813}

In [ ]:
# Calculate Meteor score
meteor_metric = Meteor()
meteor_metric.evaluate(summaries[:4], targets)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


{'meteor': 0.07293417955426351}

In [ ]:
# Calculate BleuScore
bleu_metric = Bleu()
bleu_metric.evaluate(summaries[:4], targets)

{'bleu': 0.020021970606128897}

Summarizing and evaluating class 4

In [ ]:
lexrank = model.LexRankModel(corpus4)

# Inference
summary = lexrank.summarize(corpus4)
for i in range(len(summary)):
  print("Summary Review %d: "%(i+1), summary[i])

Summary Review 1:  all the stuff i got was cool, i love the pink peace sign tongue ring :) the only complaint is that i got 2 yellow lip rings and im not much for yellow, overall very happy i got 3 belly rings 3 tongue rings 3 lip rings and a 16g cbr.
Summary Review 2:  I ended up taking mine to a silver smith and had him remove the back section of the middle ring and it really helped out with the flexibility The flaw lies in the middle part of the ring, being that its a complete ring it cuts down on the flexiblity of the over all joint.
Summary Review 3:  I ended up taking mine to a silver smith and had him remove the back section of the middle ring and it really helped out with the flexibility The flaw lies in the middle part of the ring, being that its a complete ring it cuts down on the flexiblity of the over all joint.
Summary Review 4:  Other than that a great very comfortable ring. my only wish on this ring is- I wish the cut potrion went all the way around the ring.
Summary Rev

In [ ]:
from summertime.evaluation import SUPPORTED_EVALUATION_METRICS

print(SUPPORTED_EVALUATION_METRICS)

import summertime.evaluation as st_eval

bert_eval = st_eval.BertScore()

from summertime import dataset, model
from pprint import pprint
from summertime.evaluation import BertScore
from summertime.evaluation import Meteor
from summertime.evaluation import Bleu

targets = ['I love the shipping pace.' , 'The ring itself is beautiful.' , 'I have been looking for a ring that is so beautiful and unique.' ,  'This is a solid.beautiful ring.' ,  'But if you are expecting the color in rhe picture you will be disappointed.' , 'I love this ring my only complaint is that the metal is so soft.']

summaries = summary

In [ ]:
# Calculate BertScore
bert_metric = BertScore()
bert_metric.evaluate(summaries[:4], targets)

hash_code: bert-base-uncased_L8_no-idf_version=0.3.11(hug_trans=4.5.1)


{'bert_score_f1': 0.43645626306533813}

In [ ]:
# Calculate Meteor score
meteor_metric = Meteor()
meteor_metric.evaluate(summaries[:4], targets)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


{'meteor': 0.07293417955426351}

In [ ]:
# Calculate BleuScore
bleu_metric = Bleu()
bleu_metric.evaluate(summaries[:4], targets)

{'bleu': 0.020021970606128897}

Summarizing and Evaluating class 5

In [ ]:
lexrank = model.LexRankModel(corpus5)

# Inference
summary = lexrank.summarize(corpus5)
for i in range(len(summary)):
  print("Summary Review %d: "%(i+1), summary[i])

Summary Review 1:  I have had a lot of compliments on my new ring". Perfect fit!
Summary Review 2:  The ring shines perfectly I love this ring! This ring is perfect I say why spend thousands when you don't have to?
Summary Review 3:  Thank you, Dorothy Eve's Addiction was wonderful with sending the ring and the ring is beautiful; my daughter waas thrilled with the ring.
Summary Review 4:  In the past she has had similar rings but has always been frustrated by the flimsy ring band, but this ring has a nice solid band. my girlfriend especially enjoys this ring because of the thickness of the ring band.
Summary Review 5:  I got this as my engagement ring Feb 09 This ring is beautiful and durable. I absolutely love this ring!
Summary Review 6:  Right he doesn't have to buy me an engagement ring, he can just use this as my engagement ring, as I am not into getting into debt before marriage. I love this ring because the flowers go all the way around the ring and it fits my ring finger so if 

In [ ]:
from summertime.evaluation import SUPPORTED_EVALUATION_METRICS

print(SUPPORTED_EVALUATION_METRICS)

import summertime.evaluation as st_eval

bert_eval = st_eval.BertScore()

from summertime import dataset, model
from pprint import pprint
from summertime.evaluation import BertScore
from summertime.evaluation import Meteor
from summertime.evaluation import Bleu

targets = ['This ring is gorgeous.' , 'I love the ring and would recommend it to anyone who likes to wear a ring.' , 'I am a huge fan of the ring and I would recommend it to anyone looking for a ring.' , 'I am so glad I bought this ring. ']

summaries = summary

[<class 'summertime.evaluation.bertscore_metric.BertScore'>, <class 'summertime.evaluation.bleu_metric.Bleu'>, <class 'summertime.evaluation.rouge_metric.Rouge'>, <class 'summertime.evaluation.rougewe_metric.RougeWe'>, <class 'summertime.evaluation.meteor_metric.Meteor'>]


In [ ]:
# Calculate BertScore
bert_metric = BertScore()
bert_metric.evaluate(summaries[:4], targets)

hash_code: bert-base-uncased_L8_no-idf_version=0.3.11(hug_trans=4.5.1)


{'bert_score_f1': 0.5023539066314697}

In [ ]:
# Calculate Meteor score
meteor_metric = Meteor()
meteor_metric.evaluate(summaries[:4], targets)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


{'meteor': 0.048885118917052925}

In [ ]:
# Calculate BleuScore
bleu_metric = Bleu()
bleu_metric.evaluate(summaries[:4], targets)

{'bleu': 0.024048090114825196}

Neural Model Longformer


In [ ]:
# Longformer2Roberta
longformer = model.LongformerModel()

longformer_summary = longformer.summarize(corpus2)
for i in range(len(longformer_summary)):
  print("\nSummary Review %d: "%(i+1), longformer_summary[i])

You are using a model of type encoder_decoder to instantiate a model of type encoder-decoder. This is not supported for all configurations of models and can yield errors.


Some weights of the model checkpoint at patrickvonplaten/longformer2roberta-cnn_dailymail-fp16 were not used when initializing EncoderDecoderModel: ['decoder.roberta.pooler.dense.weight', 'decoder.roberta.pooler.dense.bias']
- This IS expected if you are initializing EncoderDecoderModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing EncoderDecoderModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Longformer model: processing document of tensor([28]) tokens


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Longformer model: processing document of tensor([67]) tokens


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Longformer model: processing document of tensor([26]) tokens


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Longformer model: processing document of tensor([19]) tokens

Summary Review 1:  The ring is beyond gorgeous, I just love it.
The ring has beyond gorgeous and I just like it.22.
I wanted to know if this ring is like 2 rings in one.
This ring is more gorgeous, and I love it. I just loved it.

Summary Review 2:  The ring was not worth the postage than the original ring.
It was not even worth postage.
The ring cost more for postage than what the ring cost.
Ring was not returned as you would pay more for a ring.22 was missing.
But it was not a cheap one.

Summary Review 3:  Wore a toe ring one day and POOF the stone was gone.
It sits in a pile of broken sterling silver rings and things that belonged to broken sterling.
The stone was lost so it sits in the pile of other broken sterling rings.
POOF is now in a heap of other shattered sterling silver.

Summary Review 4:  Ring is way too small and looks like a toy when putting it on.
Ring is too small to be worn by a 1/2 carat ringat ring.
I w

In [ ]:
from summertime.evaluation import SUPPORTED_EVALUATION_METRICS

print(SUPPORTED_EVALUATION_METRICS)

import summertime.evaluation as st_eval

bert_eval = st_eval.BertScore()

from summertime import dataset, model
from pprint import pprint
from summertime.evaluation import BertScore
from summertime.evaluation import Meteor
from summertime.evaluation import Bleu

targets = ['It was not worth returning it as you would pay more for postage than what the ring cost.' ,'The ring was nice and looked like picture but had a crack in one Garnet and another one had a large chip.' , 'I would recommend you spend it a little bit more and buy a better looking ring.' , 'I\'m very disappointed with amazon for the lack of description.']

summaries = longformer_summary

[<class 'summertime.evaluation.bertscore_metric.BertScore'>, <class 'summertime.evaluation.bleu_metric.Bleu'>, <class 'summertime.evaluation.rouge_metric.Rouge'>, <class 'summertime.evaluation.rougewe_metric.RougeWe'>, <class 'summertime.evaluation.meteor_metric.Meteor'>]


In [ ]:
# Calculate BertScore
bert_metric = BertScore()
bert_metric.evaluate(summaries[:4], targets)

hash_code: bert-base-uncased_L8_no-idf_version=0.3.11(hug_trans=4.5.1)


{'bert_score_f1': 0.43079328536987305}

In [ ]:
# Calculate Meteor score
meteor_metric = Meteor()
meteor_metric.evaluate(summaries[:4], targets)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


{'meteor': 0.040066490721065565}

In [ ]:
# Calculate BleuScore
bleu_metric = Bleu()
bleu_metric.evaluate(summaries[:4], targets)

{'bleu': 0.00928011380084191}

The summarizations scored adequate scores in BertScore evaluation but very low scores in the other evaluation metrics, Meteor and BleuScore. This might be due to the fact that the texts that are being summarized are short.